In [ ]:
import sys
import os
import pandas as pd

project_root = os.path.dirname(os.path.abspath(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from src.data_processing.resultMetaUtils import frameFromResultMeta

In [19]:
df = frameFromResultMeta()
display(df.head)

<bound method NDFrame.head of      run_id  accuracy  ... meta.batchSize meta.scaling
0  WHZnBvTh   0.33945  ...             64         True
1  ZQsn4iOX   0.44825  ...             64         True

[2 rows x 15 columns]>